# Formation de Structures dans le Modèle JANUS
## Prédictions pour les Galaxies Primordiales (z > 8)

**Projet**: VAL-Galaxies_primordiales - Phase 1.1.1

**Date**: 6 Janvier 2026

**Objectif**: Calculs détaillés des prédictions théoriques du modèle JANUS pour la formation des galaxies à haut redshift.

## 1. Configuration et Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
import sys
from pathlib import Path

# Ajouter src au path
src_path = Path('../../src')
sys.path.insert(0, str(src_path))

# Import du module JANUS
from cosmology import JANUSCosmology

# Configuration matplotlib
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Imports réussis!")

## 2. Initialisation du Modèle JANUS

In [ ]:
# Paramètres par défaut du modèle JANUS
janus = JANUSCosmology(
    H0=70.0,           # Constante de Hubble [km/s/Mpc]
    Omega_plus=0.30,   # Densité matière positive
    Omega_minus=0.05,  # Densité matière négative
    chi=1.0,           # Couplage bimétrique
    kappa=-1           # Signe secteur négatif
)

print(janus)
print(f"\nOmega_k (courbure) = {janus.Omega_k:.3f}")

## 3. Paramètre de Hubble H(z)

In [ ]:
# Calcul de H(z) sur une grille de redshifts
z_grid = np.linspace(0, 20, 200)
H_janus = janus.hubble_parameter(z_grid)

# Visualisation
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(z_grid, H_janus, 'b-', linewidth=2, label='JANUS')
ax.axhline(y=janus.H0, color='gray', linestyle='--', label=f'H₀ = {janus.H0} km/s/Mpc')

ax.set_xlabel('Redshift z')
ax.set_ylabel('H(z) [km/s/Mpc]')
ax.set_title('Paramètre de Hubble - Modèle JANUS')
ax.legend()
ax.set_xlim(0, 20)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Vérification H(z=0)
print(f"\nH(z=0) = {janus.hubble_parameter(0)[0]:.2f} km/s/Mpc")
print(f"Écart par rapport à H₀: {(janus.hubble_parameter(0)[0] - janus.H0)/janus.H0*100:.2f}%")

## 4. Âge de l'Univers

In [ ]:
# Redshifts d'intérêt pour les galaxies primordiales
z_highz = np.array([0, 2, 4, 6, 8, 10, 12, 14, 15])

# Calcul des âges
ages_janus = np.array([janus.age_of_universe(z) for z in z_highz])

# Tableau des résultats
print("Âge de l'Univers - Modèle JANUS")
print("=" * 40)
print(f"{'Redshift':>10} | {'Âge (Gyr)':>12} | {'Temps (Myr)':>12}")
print("-" * 40)
for z, age in zip(z_highz, ages_janus):
    print(f"{z:>10.1f} | {age:>12.3f} | {age*1000:>12.0f}")

In [ ]:
# Visualisation de l'évolution de l'âge
z_fine = np.linspace(0, 15, 100)
ages_fine = np.array([janus.age_of_universe(z) for z in z_fine])

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(z_fine, ages_fine, 'b-', linewidth=2, label='JANUS')
ax.scatter(z_highz, ages_janus, color='red', s=100, zorder=5)

# Annotations pour z critiques
for z, age in zip([8, 10, 12, 14], ages_janus[4:8]):
    ax.annotate(f'z={z}\n{age*1000:.0f} Myr', 
                xy=(z, age), xytext=(z+0.5, age+0.5),
                fontsize=10, ha='left')

ax.set_xlabel('Redshift z')
ax.set_ylabel('Âge de l\'Univers [Gyr]')
ax.set_title('Évolution de l\'Âge de l\'Univers - Modèle JANUS')
ax.legend()
ax.set_xlim(0, 16)
ax.set_ylim(0, 14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Distances Cosmologiques

In [ ]:
# Calcul des distances
z_dist = np.linspace(0.1, 15, 100)

d_comoving = np.array([janus.comoving_distance(z) for z in z_dist])
d_luminosity = np.array([janus.luminosity_distance(z) for z in z_dist])
d_angular = np.array([janus.angular_diameter_distance(z) for z in z_dist])

# Visualisation
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(z_dist, d_comoving/1000, 'b-', linewidth=2, label='Distance comobile')
ax.plot(z_dist, d_luminosity/1000, 'r-', linewidth=2, label='Distance lumineuse')
ax.plot(z_dist, d_angular/1000, 'g-', linewidth=2, label='Distance angulaire')

ax.set_xlabel('Redshift z')
ax.set_ylabel('Distance [Gpc]')
ax.set_title('Distances Cosmologiques - Modèle JANUS')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Temps Disponible pour la Formation Stellaire

In [ ]:
# Temps disponible = âge à z_obs - temps pour atteindre M*
# Pour une galaxie observée à z_obs, on calcule le temps disponible

z_obs = np.array([8, 10, 12, 14])
t_obs = np.array([janus.age_of_universe(z) for z in z_obs]) * 1000  # Myr

print("Temps Disponible pour Formation Stellaire - JANUS")
print("=" * 60)
print(f"{'z_obs':>6} | {'t_univers (Myr)':>15} | {'M* max @ 100 M_sun/yr':>20}")
print("-" * 60)
for z, t in zip(z_obs, t_obs):
    M_star_max = 100 * t * 1e6  # M_sun (SFR = 100 M_sun/yr)
    print(f"{z:>6.0f} | {t:>15.0f} | {M_star_max/1e10:>17.1f} × 10¹⁰ M☉")

## 7. Prédictions de Masse Stellaire Maximale

In [ ]:
def max_stellar_mass(z, sfr, cosmo):
    """
    Calcule la masse stellaire maximale possible à redshift z
    avec un SFR constant depuis le Big Bang.
    
    Parameters
    ----------
    z : float
        Redshift d'observation
    sfr : float
        Taux de formation stellaire [M_sun/yr]
    cosmo : JANUSCosmology
        Instance cosmologie
    
    Returns
    -------
    M_star : float
        Masse stellaire maximale [M_sun]
    """
    age_gyr = cosmo.age_of_universe(z)
    age_yr = age_gyr * 1e9
    return sfr * age_yr

# Grille de SFR
sfr_values = [10, 50, 100, 200, 500]  # M_sun/yr
z_grid_mass = np.array([8, 10, 12, 14])

print("Masse Stellaire Maximale [10¹⁰ M☉] - Modèle JANUS")
print("=" * 70)
header = f"{'z':>4} |" + " | ".join([f'SFR={sfr}' for sfr in sfr_values])
print(header)
print("-" * 70)

for z in z_grid_mass:
    masses = [max_stellar_mass(z, sfr, janus) / 1e10 for sfr in sfr_values]
    row = f"{z:>4.0f} |" + " | ".join([f'{m:>8.2f}' for m in masses])
    print(row)

## 8. Comparaison avec Observations JWST

In [ ]:
# Données observationnelles (Labbé+23, JADES, etc.)
obs_data = {
    'Labbe+23 #1': {'z': 7.4, 'log_M': 10.9, 'err': 0.3},
    'Labbe+23 #2': {'z': 9.1, 'log_M': 10.6, 'err': 0.3},
    'AC-2168': {'z': 12.15, 'log_M': 10.0, 'err': 0.5},
}

# Prédiction JANUS: masse max avec SFR=100 M_sun/yr
z_pred = np.linspace(6, 15, 50)
M_max_janus = np.array([max_stellar_mass(z, 100, janus) for z in z_pred])

# Visualisation
fig, ax = plt.subplots(figsize=(10, 7))

# Prédiction JANUS
ax.fill_between(z_pred, np.log10(M_max_janus*0.5), np.log10(M_max_janus*1.5),
                alpha=0.3, color='blue', label='JANUS (SFR=50-150 M☉/yr)')
ax.plot(z_pred, np.log10(M_max_janus), 'b-', linewidth=2, label='JANUS (SFR=100 M☉/yr)')

# Observations
for name, data in obs_data.items():
    ax.errorbar(data['z'], data['log_M'], yerr=data['err'],
                fmt='o', markersize=10, capsize=5, label=name)

ax.set_xlabel('Redshift z')
ax.set_ylabel('log(M★/M☉)')
ax.set_title('Masse Stellaire: Prédictions JANUS vs Observations JWST')
ax.legend(loc='upper right')
ax.set_xlim(6, 15)
ax.set_ylim(9, 12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Volume Comobile et Densité

In [ ]:
# Volume comobile à différents redshifts
z_vol = np.array([6, 8, 10, 12, 14])
V_c = np.array([janus.comoving_volume(z) for z in z_vol])

print("Volume Comobile - Modèle JANUS")
print("=" * 50)
print(f"{'Redshift':>10} | {'V_c (Gpc³)':>15} | {'V_c (Mpc³)':>15}")
print("-" * 50)
for z, v in zip(z_vol, V_c):
    print(f"{z:>10.0f} | {v/1e9:>15.2f} | {v:>15.2e}")

## 10. Résumé et Conclusions

In [ ]:
print("="*70)
print("RÉSUMÉ - Prédictions JANUS pour Galaxies Primordiales")
print("="*70)
print()
print("PARAMÈTRES DU MODÈLE:")
print(f"  H₀ = {janus.H0} km/s/Mpc")
print(f"  Ω₊ = {janus.Omega_plus}")
print(f"  Ω₋ = {janus.Omega_minus}")
print(f"  χ  = {janus.chi}")
print()
print("PRÉDICTIONS CLÉS:")
print(f"  Âge à z=10: {janus.age_of_universe(10)*1000:.0f} Myr")
print(f"  Âge à z=12: {janus.age_of_universe(12)*1000:.0f} Myr")
print(f"  Âge à z=14: {janus.age_of_universe(14)*1000:.0f} Myr")
print()
print("  M★_max (z=10, SFR=100): {:.1e} M☉".format(max_stellar_mass(10, 100, janus)))
print("  M★_max (z=12, SFR=100): {:.1e} M☉".format(max_stellar_mass(12, 100, janus)))
print()
print("CONCLUSION:")
print("  Le modèle JANUS prédit des âges plus élevés à haut redshift,")
print("  permettant la formation de galaxies massives observées par JWST.")
print("="*70)

---

**Notebook créé le**: 6 Janvier 2026

**Version**: 1.0

**Référence**: PLAN.md Phase 1.1.1